In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [3]:
!pip install "gluonts>=0.12.0"
!pip install pytorch-lightning
!pip install torch

In [5]:
import pandas as pd
import numpy as np

In [7]:
# Sentinelles ILI From France (1984-2025)

# Replace this URL with your Dropbox sharing link
url_deepar = 'https://www.dropbox.com/scl/fi/bn8q34o3f0jki4pp8cwkq/sentinelle_ILI_France_1984_2025.csv?rlkey=h57lvdkvvmpoifwprbl6nc4l2&st=m1r7lprx&dl=1'

# Read the CSV file directly from the URL
sentinelle_ili_france_1984_2025 = pd.read_csv(url_deepar)


In [9]:
####################################### ILI ########################################
# Drop the 'Unnamed: 0' column
ili_deepar = sentinelle_ili_france_1984_2025.copy()
ili_deepar = ili_deepar.drop('Unnamed: 0', axis=1)

# Rename columns
ili_deepar = ili_deepar.rename(columns={'date': 'timestamp', 'inc': 'target'})

# Add item_id column with 'ILI' value
ili_deepar['item_id'] = 'ILI'

#Preview
ili_deepar.head()

,timestamp,target,item_id
0,1984-10-29,68422.0,ILI
1,1984-11-05,135223.0,ILI
2,1984-11-12,87330.0,ILI
3,1984-11-19,72029.0,ILI
4,1984-11-26,78620.0,ILI


In [11]:
# Input.csv

# Read the CSV file into a DataFrame
url_input = 'https://www.dropbox.com/scl/fi/f59qxf16fdyexxwelry17/Inputs.csv?rlkey=z1ke2dm733puv20d3crdxrfad&st=af9d89bd&dl=1'

input = pd.read_csv(url_input, sep = ';')

# Display the first few rows to verify the import
input.head()

,Data,Scenario,dstart_season_test,dstart_test,dend_test
0,ILI,3_seasons_01_10,2016-07-18,2016-10-03,2019-09-02
1,ILI,3_seasons_01_11,2016-07-18,2016-11-07,2019-09-02
2,ILI,3_seasons_01_12,2016-07-18,2016-12-05,2019-09-02
3,ILI,3_seasons_15_12,2016-07-18,2016-12-19,2019-09-02
4,ILI,4_weeks_M4_2016,2016-07-18,2016-12-19,2017-01-09


In [13]:
input

,Data,Scenario,dstart_season_test,dstart_test,dend_test
0,ILI,3_seasons_01_10,2016-07-18,2016-10-03,2019-09-02
1,ILI,3_seasons_01_11,2016-07-18,2016-11-07,2019-09-02
2,ILI,3_seasons_01_12,2016-07-18,2016-12-05,2019-09-02
3,ILI,3_seasons_15_12,2016-07-18,2016-12-19,2019-09-02
4,ILI,4_weeks_M4_2016,2016-07-18,2016-12-19,2017-01-09
...,...,...,...,...,...
270,Dengue,pic_M1_2017,2017-09-04,2018-04-01,2018-09-03
271,Dengue,pic_M4_2018,2018-09-03,2019-03-17,2019-10-07
272,Dengue,pic_M3_2018,2018-09-03,2019-03-24,2019-10-07
273,Dengue,pic_M2_2018,2018-09-03,2019-03-31,2019-10-07


In [15]:
input_df = input[input['Data'] == 'ILI']
# end_scenario = "4_weeks_P4_2018"
#mask = input_df # ['Scenario'] == end_scenario
#print(f"Found matches: {mask.sum()}")  # Should be 1
input_ili = input_df
input_ili

,Data,Scenario,dstart_season_test,dstart_test,dend_test
0,ILI,3_seasons_01_10,2016-07-18,2016-10-03,2019-09-02
1,ILI,3_seasons_01_11,2016-07-18,2016-11-07,2019-09-02
2,ILI,3_seasons_01_12,2016-07-18,2016-12-05,2019-09-02
3,ILI,3_seasons_15_12,2016-07-18,2016-12-19,2019-09-02
4,ILI,4_weeks_M4_2016,2016-07-18,2016-12-19,2017-01-09
5,ILI,4_weeks_M3_2016,2016-07-18,2016-12-26,2017-01-16
6,ILI,4_weeks_M2_2016,2016-07-18,2017-01-02,2017-01-23
7,ILI,4_weeks_M1_2016,2016-07-18,2017-01-09,2017-01-30
8,ILI,4_weeks_0_2016,2016-07-18,2017-01-16,2017-02-06
9,ILI,4_weeks_P1_2016,2016-07-18,2017-01-23,2017-02-13


In [17]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator


In [120]:
def prepare_scenario_data(ili_deepar, input_ili, scenario_name):
    """
    Prepare data for a specific scenario using GluonTS's dataset splitting.
    """
    # Get scenario details
    scenario = input_ili[input_ili['Scenario'] == scenario_name].iloc[0]
    dstart_test = pd.to_datetime(scenario['dstart_test'])
    dend_test = pd.to_datetime(scenario['dend_test'])

    # Ensure timestamp is datetime and resample to weekly frequency
    ili_deepar['timestamp'] = pd.to_datetime(ili_deepar['timestamp'])
    # Resample to weekly frequency with forward fill for missing values
    ili_deepar = ili_deepar.set_index('timestamp').resample('W').ffill().reset_index()

    # Calculate prediction length (in weeks)
    prediction_length = (dend_test - dstart_test).days // 7

    # Create PandasDataset from long format
    dataset = PandasDataset.from_long_dataframe(
        ili_deepar,
        item_id='item_id',
        target='target',
        timestamp='timestamp',
        freq='W'
    )

    # Split the data
    offset = -prediction_length  # negative offset for last n points
    training_data, test_gen = split(dataset, offset=offset)
    test_data = test_gen.generate_instances(
        prediction_length=prediction_length,
        windows=1
    )

    # Convert test_data instances to DataEntry dictionaries
    test_data = [
        {'target': instance[0]['target'], 'start': instance[0]['start']}
        for instance in test_data
    ]
    test_data = ListDataset(test_data, freq=dataset.freq) # Convert to ListDataset

    # Get ground truth data for the test period
    mask = (ili_deepar['timestamp'] > dstart_test) & (ili_deepar['timestamp'] <= dend_test)
    prediction_dates = ili_deepar[mask]['timestamp'].values
    inc = ili_deepar[mask]['target'].values

    return training_data, test_data, prediction_length, inc, prediction_dates


In [122]:
def train_and_forecast(training_data, test_data, prediction_length):
    """
    Train DeepAR model and generate forecasts.
    """
    # Train the model
    model = DeepAREstimator(
        prediction_length=prediction_length,
        freq="W",
        trainer_kwargs={"max_epochs": 5}
    ).train(training_data)

    # Generate forecasts
    forecasts = list(model.predict(test_data))
    forecast = forecasts[0]

    # Extract quantiles
    y = forecast.median
    y_lo = forecast.quantile(0.1)
    y_up = forecast.quantile(0.9)

    return y, y_lo, y_up


In [124]:
def run_all_scenarios(ili_deepar, input_ili):
    """
    Run forecasting for all scenarios and compile results.
    """
    results = []

    for scenario_name in input_ili['Scenario'].unique():
        print(f"Processing scenario: {scenario_name}")

        # Prepare data for this scenario
        training_data, test_data, prediction_length, inc, prediction_dates = \
            prepare_scenario_data(ili_deepar, input_ili, scenario_name)

        # Train model and generate forecasts
        y, y_lo, y_up = train_and_forecast(
            training_data, test_data, prediction_length
        )

        # Ensure all arrays are the same length
        n = len(inc)
        y = y[:n]
        y_lo = y_lo[:n]
        y_up = y_up[:n]

        # Compile results for this scenario
        scenario_results = pd.DataFrame({
            'scenario': scenario_name,
            'date': prediction_dates,
            'inc': inc,
            'y': y,
            'y_lo': y_lo,
            'y_up': y_up
        })

        results.append(scenario_results)

    # Combine all results
    final_results = pd.concat(results, ignore_index=True)

    return final_results

In [126]:
results_df = run_all_scenarios(ili_deepar, input_ili)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Processing scenario: 3_seasons_01_10


/opt/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name  | Type        | Params | Mode  | In sizes                                                       | Out sizes    
-------------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 23.8 K | train | [[1, 1], [1, 1], [1, 308, 3], [1, 308], [1, 308], [1, 152, 3]] | [1, 100, 152]
-------------------------------------------------------------------------------------------------------------------------------
23.8 K    Trainable params
0         Non-trainable params
23.8 K    Total params
0.095     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 11.26201 (best 11.26201), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_222/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 10.65576 (best 10.65576), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_222/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 10.47962 (best 10.47962), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_222/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 10.34481 (best 10.34481), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_222/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 10.23168 (best 10.2

...

Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 11.71918 (best 11.71918), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_275/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 11.09584 (best 11.09584), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_275/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 10.91331 (best 10.91331), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_275/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 10.77558 (best 10.77558), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_275/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 10.66873 (best 10.6

Processing scenario: pic_M1_2018


Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 11.82271 (best 11.82271), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_276/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 11.12340 (best 11.12340), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_276/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 10.93428 (best 10.93428), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_276/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 10.78258 (best 10.78258), saving model to '/Users/apple/Downloads/IPLESP_SCAI_SU/FoundationModels_TimeSeries/DeepAR/lightning_logs/version_276/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 10.71238 (best 10.7

In [128]:
results_df

,scenario,date,inc,y,y_lo,y_up
0,3_seasons_01_10,2016-10-09,13379.0,21827.138672,8616.690430,40457.328125
1,3_seasons_01_10,2016-10-16,13909.0,23121.292969,8047.591309,37912.042969
2,3_seasons_01_10,2016-10-23,12920.0,20160.746094,3834.618652,34028.039062
3,3_seasons_01_10,2016-10-30,9378.0,19408.968750,6915.547363,32326.140625
4,3_seasons_01_10,2016-11-06,15145.0,15740.865234,3270.221191,30030.539062
...,...,...,...,...,...,...
1642,pic_M1_2018,2019-08-04,1944.0,61398.679688,17185.794922,124117.242188
1643,pic_M1_2018,2019-08-11,2337.0,59423.531250,5264.743164,115399.742188
1644,pic_M1_2018,2019-08-18,1592.0,55958.335938,10590.936523,94389.046875
1645,pic_M1_2018,2019-08-25,1593.0,54168.828125,19522.134766,109132.773438


In [130]:
# prompt: filter forecasts to create a new dataframe which include all the rows containing Scenario after 4_weeks_P4_2018 until end and call it forecasts_pic. Create forecast_ILI which is all entries 'Scenario' <= '4_weeks_P4_2018'

# Create forecasts_pic
forecasts_pic = results_df[
    (results_df['scenario'] > '4_weeks_P4_2018')
]

# Create forecast_ILI
forecasts_ILI = results_df[
    (results_df['scenario'] <= '4_weeks_P4_2018')
]


In [132]:
# prompt: save final_combined_df as csv

# Save the final_combined_df to a CSV file
forecasts_ILI.to_csv('forecast_ILI_5Epochs.csv', index=False)

In [134]:
forecasts_pic

,scenario,date,inc,y,y_lo,y_up
700,pic_01_10_2016,2016-10-09,13379.0,32325.527344,3789.336426,59835.785156
701,pic_01_10_2016,2016-10-16,13909.0,31868.572266,3599.947998,57716.523438
702,pic_01_10_2016,2016-10-23,12920.0,28324.609375,-6979.863281,57173.164062
703,pic_01_10_2016,2016-10-30,9378.0,32884.183594,-10605.500000,66873.492188
704,pic_01_10_2016,2016-11-06,15145.0,37944.792969,-4596.630371,81778.210938
...,...,...,...,...,...,...
1642,pic_M1_2018,2019-08-04,1944.0,61398.679688,17185.794922,124117.242188
1643,pic_M1_2018,2019-08-11,2337.0,59423.531250,5264.743164,115399.742188
1644,pic_M1_2018,2019-08-18,1592.0,55958.335938,10590.936523,94389.046875
1645,pic_M1_2018,2019-08-25,1593.0,54168.828125,19522.134766,109132.773438


In [136]:
# prompt: Look at the Scenario column, for each Scenario, find the max y value and extract the rows into a new dataframe titled pic_ili. Also give me the count of this dataframe. Remove inc column

# Find the max 'y' value for each scenario
pic_ili = forecasts_pic.loc[forecasts_pic.groupby('scenario')['y'].idxmax()]

# Remove the 'inc' column
pic_ili = pic_ili.drop('inc', axis=1)

# Display the count of the dataframe
print("Count of pic_ili dataframe:", len(pic_ili))

# Display the dataframe
pic_ili

Count of pic_ili dataframe: 24


,scenario,date,y,y_lo,y_up
720,pic_01_10_2016,2017-02-26,77000.554688,7477.997559,142497.468750
853,pic_01_10_2017,2018-05-06,43923.355469,-12032.158203,92195.945312
958,pic_01_10_2018,2018-10-07,42090.437500,-6137.750977,74533.359375
762,pic_01_11_2016,2017-05-07,53711.687500,5737.222168,101973.632812
889,pic_01_11_2017,2018-05-13,60742.601562,-1774.914917,150039.406250
1042,pic_01_11_2018,2019-07-21,33157.253906,-24156.017578,82039.164062
795,pic_01_12_2016,2017-06-11,59695.351562,6006.179199,131109.765625
911,pic_01_12_2017,2018-03-11,61256.578125,23733.902344,113807.250000
1080,pic_01_12_2018,2019-07-14,71506.531250,22023.517578,124510.718750
815,pic_15_12_2016,2017-04-30,44691.511719,11350.648438,75706.375000


In [138]:
# prompt: For pic_ili, rename some columns and the order of columns should be as follows : Data, Scenario, y=inc_pic, y_lo = inc_pic_lo, y_up = inc_pic_up, date = date_pic

# Rename columns and reorder
pic_ili = pic_ili.rename(columns={
    'date': 'date_pic',
    'y': 'inc_pic',
    'y_lo': 'inc_pic_lo',
    'y_up': 'inc_pic_up',
    'scenario': 'Scenario'
})

pic_ili = pic_ili[['Scenario', 'inc_pic', 'inc_pic_lo', 'inc_pic_up', 'date_pic']]

pic_ili


,Scenario,inc_pic,inc_pic_lo,inc_pic_up,date_pic
720,pic_01_10_2016,77000.554688,7477.997559,142497.468750,2017-02-26
853,pic_01_10_2017,43923.355469,-12032.158203,92195.945312,2018-05-06
958,pic_01_10_2018,42090.437500,-6137.750977,74533.359375,2018-10-07
762,pic_01_11_2016,53711.687500,5737.222168,101973.632812,2017-05-07
889,pic_01_11_2017,60742.601562,-1774.914917,150039.406250,2018-05-13
1042,pic_01_11_2018,33157.253906,-24156.017578,82039.164062,2019-07-21
795,pic_01_12_2016,59695.351562,6006.179199,131109.765625,2017-06-11
911,pic_01_12_2017,61256.578125,23733.902344,113807.250000,2018-03-11
1080,pic_01_12_2018,71506.531250,22023.517578,124510.718750,2019-07-14
815,pic_15_12_2016,44691.511719,11350.648438,75706.375000,2017-04-30


In [140]:
# prompt: save pic_ili as csv

pic_ili.to_csv('DeepAR_pic_ili__5Epochs.csv', index=False)
